In [16]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from Preprocessing.to_embedding import WordEmbedding
from data_format_phase3 import formatting
from Preprocessing.helper_functions import import_embedding, embedding_matrix_word2vec
from sklearn.model_selection import train_test_split

In [2]:
import keras

config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 8} ) 
config.gpu_options.allow_growth = True
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

keras.backend.tensorflow_backend._get_available_gpus()
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2475676247735203431
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4977721344
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16055511008916846413
physical_device_desc: "device: 0, name: GeForce RTX 2070, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [3]:
file_path = open("fomatted_data",'rb')
data = pickle.load(file_path)

In [4]:
data = data.drop("year", axis=1)

#data = data[:round(len(data)*.2)]
y = pd.get_dummies(data['polarity'])
X_train, X_dev, y_train, y_dev = train_test_split(data, y, test_size = 0.20, random_state=42)

X_train_nlp, X_dev_nlp = X_train['reviewText'], X_dev['reviewText']

X_train_meta, X_dev_meta = X_train.iloc[:,3:], X_dev.iloc[:,3:]
embedding_size = 300 #number of feature weights in embeddings
max_len = 400

In [5]:
#Basic Vectorization of data
#Review data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train_nlp)
word_index = tokenizer.word_index

def vectorize(data, tokenizer ,max_len):
    sequences = tokenizer.texts_to_sequences(data)
    padding = pad_sequences(sequences, maxlen = max_len)
    
    return padding

X_train_nlp = vectorize(X_train_nlp, tokenizer , max_len)
X_dev_nlp = vectorize(X_dev_nlp, tokenizer, max_len)

print('Found %s unique tokens.' % len(word_index))
print('Shape of train tensor', X_train_nlp.shape)
print('Shape of dev tensor', X_dev_nlp.shape)

Found 38806 unique tokens.
Shape of train tensor (72000, 400)
Shape of dev tensor (18000, 400)


## Game Data

In [6]:
game_df = pd.read_csv("../phase1_video_games-test-hidden.csv")
game_labels = pd.read_csv("../true_labels/true_game_labels.txt", header=None)
merged = pd.concat([game_df, game_labels], axis=1).drop('polarity', axis=1).rename(columns={0: "polarity"})



# get a list of columns

cols = list(merged)
# move the column to head of list using index, pop and insert
cols.insert(0, cols.pop(cols.index('polarity')))
cols 
merged = merged.ix[:, cols]
merged.to_csv('merged_df.csv', index=False)



from data_format_phase3 import formatting
game_data = formatting("merged_df.csv", test=True)

game_data = game_data.drop('year', axis=1)


game_y = pd.get_dummies(game_data['polarity'])




"""HVIS DU ÆNDRE HVOR MANGE FEATURES DER ER, SKAL DU ÆNDRE INDEXERING HER:"""
game_train_meta = game_data.iloc[:,3:]


game_train_nlp = game_data['summary']



game_train_nlp = vectorize(game_train_nlp, tokenizer , max_len)
game_sets = [game_train_nlp, game_train_meta]



/home/dsproj016/anaconda3/envs/py36tfnew/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  del sys.path[0]


Adding topic features
Test registered, writing topics to dataframe
Opening topic file
Adding test topics
['bad', 'funny', 'new', 'old', 'worth', 'long', 'worst', 'big', 'real', 'little', 'original', 'boring', 'excellent', 'acting', 'good', 'interesting', 'set', 'quality', 'special', 'great', 'better', 'best', 'like']


In [7]:
#game_data

#game_data.drop('affin_score', axis=1, inplace=True)


game_data



,polarity,summary,reviewText,bad,funny,new,old,worth,long,worst,...,good,interesting,set,quality,special,great,better,best,like,affin_score
0,positive,"[the, very, beginning, of, a, true, party]","[days, people, said, quot, we're, party, mario, party, quot, seems, long, past, game, first, came, seemed, thing, people, talked, yet, today, 5, mario, parties, collection, one, still, wonderful, ...",0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0.217143
1,positive,"[best, star, wars, game, on, xbox]","[game, pretty, fun, configure, controls, way, like, game, definitely, worth, buying, although, quot, puzzles, quot, game, pretty, hard, figure, sometimes, find, faq's, online, help, best, i've, ev...",1,0,1,0,1,0,0,...,1,0,0,0,0,0,0,1,1,0.218391
2,positive,"[good, story, decent, game, star, wars, fans, will, enjoy, it]","[think, star, wars, fans, pleased, game, much, better, typical, junk, star, wars, games, coming, years, that's, refreshing, everyone, like, game, think, pretty, solid, title, live, minor, annoyanc...",0,0,0,0,0,0,0,...,1,0,1,0,0,0,1,0,1,0.203390
3,positive,"[yes, mario, is, back]","[best, mario, game, almost, like, classic, super, mario, 64, except, space, new, ways, play, wii, nun, chuck, movement, makes, greatest, games, decade, yes, mario, back]",0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0.480000
4,positive,"[quite, a, remarkable, game]","[one, original, shooter, games, released, dripping, atmosphere, also, horror, parts, make, jump, like, post, apocalyptic, world, trying, survive, belongs, creme, far, first, person, shooters, goes...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0.081967
5,positive,"[swords, and, bows, and, nukes, oh, my]","[okay, frank, game, owns, like, fft, permanent, death, thing, ffta, die, die, die, die, permanently, graphics, wise, game, dumbed, fft, class, look, good, far, better, fft, quite, repetitive, that...",0,1,0,1,1,0,0,...,1,0,0,0,0,0,1,0,1,0.000000
6,positive,"[a, work, of, art]","[game, great, action, mean, look, screenshots, know, looks, strange, since, use, lines, guide, like, thought, ahead, buy, different, skins, line, anyway, point, make, long, story, short, turns, ki...",0,0,1,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0.105263
7,negative,"[what, controls, and, not, resident, evil]","[great, expectations, first, saw, videos, heard, news, new, resident, evil, game, huge, fan, series, especially, re2, code, veronica, best, time, 4, already, disappointing, years, ago, action, puz...",0,0,1,0,0,1,0,...,0,0,0,0,0,1,1,1,1,-0.141762
8,positive,"[great, buy]","[let, first, say, ever, regret, buying, game, people, complain, weird, 3rd, person, view, think, beats, even, first, person, shooter, view, especially, psp, perfect, portable, game, missions, espe...",0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0.129630
9,positive,"[finally, redemption]","[played, first, call, hard, pressed, believe, games, created, next, generation, first, real, last, gen, feel, graphically, felt, pretty, horrible, overall, bound, blood, red, dead, revolver, mind,...",0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,1,1,0.126168


## Movie Data

In [8]:
movie_df = pd.read_csv("../phase1_movie_reviews-test-hidden.csv")
movie_labels = pd.read_csv("../true_labels/true_movie_labels.txt", header=None)
merged = pd.concat([movie_df, movie_labels], axis=1).drop('polarity', axis=1).rename(columns={0: "polarity"})
# get a list of columns
cols = list(merged)
# move the column to head of list using index, pop and insert
cols.insert(0, cols.pop(cols.index('polarity')))
cols

# use ix to reorder
merged = merged.ix[:, cols]
merged.to_csv('merged_df.csv', index=False)
from data_format_phase3 import formatting
movie_data = formatting("merged_df.csv", test=True)

movie_data = movie_data.drop('year', axis=1)


movie_y = pd.get_dummies(movie_data['polarity'])



"""HVIS DU ÆNDRE HVOR MANGE FEATURES DER ER, SKAL DU ÆNDRE INDEXERING HER:"""
movie_train_meta = movie_data.iloc[:,3:]


movie_train_nlp = movie_data['summary']

movie_train_nlp = vectorize(movie_train_nlp, tokenizer , max_len)
movie_sets = [movie_train_nlp, movie_train_meta]







/home/dsproj016/anaconda3/envs/py36tfnew/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # This is added back by InteractiveShellApp.init_path()


Adding topic features
Test registered, writing topics to dataframe
Opening topic file
Adding test topics
['bad', 'funny', 'new', 'old', 'worth', 'long', 'worst', 'big', 'real', 'little', 'original', 'boring', 'excellent', 'acting', 'good', 'interesting', 'set', 'quality', 'special', 'great', 'better', 'best', 'like']


In [9]:
from keras.models import load_model
# Returns a compiled model identical to the previous one
model = load_model('funct_GRU_model_ablation_no_reviewText.h5')


In [10]:
predicted_game = model.predict(x=game_sets,batch_size=200, verbose=1)


#ARGMAX PREDICTIONS GAME
for i in predicted_game:
    max_idx = np.argmax(i)
    if max_idx == 0:
        i[0] = 1
        i[1] = 0
    else:
        i[1] = 1
        i[0] = 0
        
        


predicted_movie = model.predict(x=movie_sets,batch_size=200, verbose=1)
        
#ARGMAX PREDICTIONS MOVIE
for i in predicted_movie:
    max_idx = np.argmax(i)
    if max_idx == 0:
        i[0] = 1
        i[1] = 0
    else:
        i[1] = 1
        i[0] = 0
        


10000/10000 [==============================] - 6s 590us/step


In [11]:
game_y = game_y.as_matrix()
print(game_y)
print(predicted_game)

print(type(game_y))
print(type(predicted_game))

[[0 1]
 [0 1]
 [0 1]
 ...
 [1 0]
 [1 0]
 [1 0]]
[[1. 0.]
 [0. 1.]
 [1. 0.]
 ...
 [0. 1.]
 [1. 0.]
 [0. 1.]]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


/home/dsproj016/anaconda3/envs/py36tfnew/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [12]:
movie_y = movie_y.as_matrix()
print(movie_y)
print(predicted_movie)

print(type(game_y))
print(type(predicted_movie))

[[0 1]
 [0 1]
 [0 1]
 ...
 [1 0]
 [1 0]
 [1 0]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [1. 0.]]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


/home/dsproj016/anaconda3/envs/py36tfnew/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


## Accuracy score Game data: 


In [13]:
from sklearn.metrics import accuracy_score



In [39]:
y_pred_game = predicted_game
y_true_game = game_y
accuracy_score(y_true_game, y_pred_game)

0.6105382650647999

## Accuracy score movie data: 

In [44]:
y_pred_movie = predicted_movie
y_true_movie = movie_y
accuracy_score(y_true_movie, y_pred_movie)

0.6151

## Precision, Recall, f1-score for Game Data

In [48]:
from sklearn.metrics import classification_report

(0.6204471000594747, 0.6105382650647999, 0.6023600934454727, None)
(0.6105382650647999, 0.6105382650647999, 0.6105382650647999, None)
(0.6204471000594746, 0.6105382650647999, 0.6023600934454727, None)


In [49]:
target_names = ['class 0', 'class 1']
print(classification_report(game_y, predicted_game, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.65      0.47      0.55     10571
     class 1       0.59      0.75      0.66     10571

   micro avg       0.61      0.61      0.61     21142
   macro avg       0.62      0.61      0.60     21142
weighted avg       0.62      0.61      0.60     21142
 samples avg       0.61      0.61      0.61     21142



## Precision, Recall, f1-score for Movie Data

In [50]:
target_names = ['class 0', 'class 1']
print(classification_report(movie_y, predicted_movie, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.65      0.48      0.55      4973
     class 1       0.59      0.75      0.66      5027

   micro avg       0.62      0.62      0.62     10000
   macro avg       0.62      0.61      0.61     10000
weighted avg       0.62      0.62      0.61     10000
 samples avg       0.62      0.62      0.62     10000

